<a href="https://colab.research.google.com/github/olaviinha/NeuralImageGeneration/blob/main/BigSleep_lucidrains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Big Sleep <font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">BigGAN+CLIP: Neural text-to-image</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralImageGeneration" target="_blank"><font color="#999" size="4">Github</font></a>

Big Sleep generates images from text input. It's a combination of [CLIP](https://github.com/openai/CLIP) by OpenAI and [BigGAN](https://arxiv.org/abs/1809.11096) by Andrew Brock et al., a concept introduced by [Ryan Murdock](https://github.com/rynmurdock) in his [original notebook](https://colab.research.google.com/drive/1NCceX2mbiKOSlAd_o7IU7nA9UskKN5WR?usp=sharing). This notebook is for [my modified fork](https://github.com/olaviinha/big-sleep) of [Big Sleep implementation](https://github.com/lucidrains/big-sleep) by [Phil Wang](https://github.com/lucidrains). This notebook will produce only 512x512 px imagery.

<hr size="1" color="#666">

### Tips
- Enter a simple string of text to `generate_image_of` field. You may also use a semicolon `;` as a separator to batch process multiple strings of texts to images in one go, and/or pipe `|` to train the image on multiple strings of text.
- Enter `output_dir` path relative to your Google Drive root, or leave blank to not save output anywhere outside this notebook.
- In vast majority of cases, over 400 iterations seems to be a waste of time.

In [ ]:
#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.

force_setup = False
pip_packages = ''
main_repository = ''

import os
from google.colab import output
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if not os.path.isdir('/content/drive') and force_setup == False:
  from google.colab import drive
  drive.mount('/content/drive')

# Drive symlink
if not os.path.isdir('/content/mydrive') and force_setup == False:
  os.symlink('/content/drive/My Drive', '/content/mydrive')
  drive_root_set = True
drive_root = '/content/mydrive/'

!pip install git+https://github.com/olaviinha/big-sleep.git

from tqdm.notebook import tqdm, trange
from IPython.display import Image, display
from big_sleep import Imagine
from PIL import Image as pimage

# !git clone {main_repository}
dir_tmp = '/content/tmp/'
dir_steps = '/content/steps/'
create_dirs([dir_tmp, dir_steps])
%cd {dir_tmp}

output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@markdown <br>

#@markdown #S̨̛̤͕ ̢̩̫̥͐ ̗͈̎l̬̀͌͑͠ ̺͚̼̻̌̃ ̱̘̘̘̃̒̚͝e̦̪̹͂ ̖̣͖̗̆̓͘ ̛̟̝̆ȩ̥͍̘̅͌̕ ̟̐̔̉͂ ̗̫͗̈́p̗̔̎̕

#@markdown <br>

generate_image_of = "" #@param {type:"string"}
output_dir = '' #@param {type:"string"}

#@markdown <hr color="#666" size="1">
#@markdown <font size="1">&nbsp;</font>

#@markdown ### Advanced settings

iterations = 400 #@param {type:"slider", min:0, max:500, step:10}
save_every = 50 #@param {type:"slider", min:0, max:100, step:1}
display_save_every = True #@param {type:"boolean"}
# save_progress = True #@param {type:"boolean"}
save_progress = True
#learning_rate = 5e-2 #@param {type:"number"}
max_classes = 0 #@param {type:"slider", min:0, max:100, step:1}
create_video = False #@param {type:"boolean"}
# SEED = 553905700049900 #@param {type: "number"}
learning_rate = 5e-2

# save_all_steps = True #@param {type:"boolean"}
save_all_steps = True

if not save_all_steps:
  clean_dirs([dir_tmp, dir_steps])

uniq_id = gen_id()

op(c.title, 'Majik & wizardry...\n')

if output_dir is '' or not output_dir:
  drive_root = '/content/fauxdrive/'
  output_dir = 'output'
  create_dirs([drive_root, output_dir])

text = generate_image_of
if text is not '':
  if ";" in text:
    texts = text.split(';')
    texts = [text.strip() for text in texts]
  else:
    texts = [text]
else:
  text = requests.get('https://api.inha.asia/k/?nouns=2&space=true&capitalize=true').text
  texts = [text]

MAX_CLASSES = max_classes
if max_classes is 0:
  MAX_CLASSES = None

SAVE_EVERY = 1
SAVE_PROGRESS = save_progress
LEARNING_RATE = learning_rate
ITERATIONS = iterations
SEED = 0 #random.seed(5)

for texte in texts:

  
  if not save_all_steps:
    clean_dirs([dir_tmp, dir_steps])

  output.clear()
  display_text = texte
  op(c.okb, '\nGenerating image of '+display_text)
  op(c.okb, 'Run ID:', uniq_id)
  op(c.title, 'Sweet dreams.\n')

  title = texte.split("|")[0].title()
  file_title = ''.join(e for e in title if e.isalnum())
  id = uniq_id+'_'+file_title
  #texte = [frase.strip() for frase in texte.split("|")]

  dir_output = fix_path(drive_root+output_dir)+id+'/'
  dir_progress = dir_output+'progress/'
  create_dirs([dir_output, dir_progress])

  if save_all_steps:
    dir_steps = dir_output+'steps/'
    create_dirs([dir_steps])

  model = Imagine(
      text = texte,
      save_every = 1,
      lr = LEARNING_RATE,
      iterations = ITERATIONS+1,
      save_progress = SAVE_PROGRESS,
      seed = SEED,
      max_classes = MAX_CLASSES
  )

  every = 0
  try:
    for epoch in trange(1, desc = 'epochs'):
      for i in trange(model.iterations, desc = 'iteration'):
        model.train_step(epoch, i)

        # if i == 0 or i % model.save_every != 0:
        #   continue

        filename = texte.replace(' ', '_')
        filename = filename.replace('-', '_')
        filename = filename.replace('|', '--')
        # if i == 0 or i % save_every != 0:
        xfile = './'+filename+'.'+str(i)+'.png'
        step_file = dir_steps+str(i).zfill(4)+'.png'
        !cp {xfile} {step_file}
        if i == 0 or every is save_every:
          # print('saving save_every file')
          image = Image(f'./{filename}.png')
          out = dir_progress+filename+'_'+str(i).zfill(4)+'.png'
          display_out = out.replace(drive_root, '')
          !cp {filename}.png {out}
          display(image)
          op(c.ok, '^Image saved as', display_out+'\n\n')
          every = 0
        if i == iterations:
          break
        every += 1
  except KeyboardInterrupt:
      pass

  last_step = dir_steps+str(i).zfill(4)+'.png'
  fin_out = dir_output+file_title+'.png'
  !cp {last_step} {fin_out}
  display_fin = fin_out.replace(drive_root, '')
  op(c.ok, 'Final image saved as', display_fin)

  if create_video is True:

    op(c.title, '\nGenerating video\n')

    init_frame = 1
    last_frame = iterations

    fps = 30
    output_video = dir_output+file_title+'.mp4'
    frames = []

    for i in range(init_frame,last_frame): #
      filename = f"{dir_steps}/{i:04}.png"
      frames.append(pimage.open(filename))

    from subprocess import Popen, PIPE
    p = Popen(['ffmpeg', '-y', '-f', 'image2pipe', '-vcodec', 'png', '-r', str(fps), '-i', '-', '-vcodec', 'libx264', '-r', str(fps), '-pix_fmt', 'yuv420p', '-crf', '13', '-preset', 'veryslow', output_video], stdin=PIPE)
    for im in tqdm(frames):
      im.save(p.stdin, 'PNG')
    p.stdin.close()
    p.wait()

    #---------------

    fin_vid = fin_out.replace('.png', '.mp4')
    op(c.ok, 'Video saved as', fin_vid)

op(c.title, '\nFIN.')
